In [2]:

import os
os.chdir('/root/MaskCLIP')
import sys
sys.path.append('/root/MaskCLIP/')
sys.path.append('/root/MaskCLIP/mmseg')
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.apis import single_gpu_test
import argparse
from mmcv.utils import Config, DictAction, get_git_hash
from mmcv.cnn.utils import revert_sync_batchnorm
import mmcv
#from mmseg.utils import get_device

import os.path as osp

parser = argparse.ArgumentParser(description='Train a segmentor')

parser.add_argument('--config', default='configs/maskclip_plus/zero_shot/maskclip_plus_r50_deeplabv2_r101-d8_512x512_20k_voc12aug_20.py',\
     help='test config file path')
parser.add_argument(
    '--load-from', default='test_outs/latest.pth', help='the checkpoint file to load weights from')
args = parser.parse_args()
cfg = Config.fromfile(args.config)

# Changes to switch to our RGB-S data
cfg.data.train.img_dir = 'RGB_S_Images/train'
cfg.data.train.ann_dir = 'RGB_S_Annotations/train'
cfg.data.train.split = 'ImageSets/Segmentation/RGBS_train.txt'
cfg.data.train.type = 'PascalVOCDatasetRGBS'

# Normalization in 4D
# Set mean to 0.5, std 0.5 (0,1) range of data for saliency
#
cfg.data.train['pipeline'][6] = \
    {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53, 0.5], 'std': [58.395, 57.12, 57.375, 0.5], 'to_rgb': True}

# Remove unnecessary augmentations (changes shape) for now
cfg.data.train['pipeline'].pop(5) # PhotoMetricDistortion

# Set parameters for validation data
cfg.data.val.img_dir = 'RGB_S_Images/val'
cfg.data.val.ann_dir = 'RGB_S_Annotations/val'
cfg.data.val.split = 'ImageSets/Segmentation/RGBS_val.txt'
cfg.data.val.type = 'PascalVOCDatasetRGBS'

# Set normalization in 4D for val
cfg.data.val['pipeline'][1]['transforms'][2] = \
    {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53, 0.5], 'std': [58.395, 57.12, 57.375, 0.5], 'to_rgb': True}

# Set batch size to 2 to allow batch norm in ASPP decoder head to work
cfg.data['samples_per_gpu'] = 12

## Remove reduce zero label for our binary mask, class agnostic training  
# Otherwise produces bug with 0/255 rolled back labels
cfg.data.train['pipeline'][1]['reduce_zero_label'] = False
# Find where to do so for val too

# Remove some augmentations
#cfg.data.train['pipeline'][2] = {'type': 'Resize', 'img_scale': (512, 512), 'ratio_range': (1.0, 1.0)}
#cfg.data.train['pipeline'][3] = {'type': 'RandomCrop', 'crop_size': (512, 512), 'cat_max_ratio': 1.0}
#cfg.data.train['pipeline'][4] = {'type': 'RandomFlip', 'prob': 0.0}
# Change hook to tensorboard
cfg.log_config = dict(
interval=50,
hooks=[
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')
])

## Add in loading annotatio[15, 16, 17, 18, 19]
cfg.data.val['pipeline'].append({})
cfg.data.val['pipeline'][2] = cfg.data.val['pipeline'][1]
cfg.data.val['pipeline'][1] =\
    {'type': 'LoadAnnotations', 'reduce_zero_label': False, 'suppress_labels': []}

## Make sure it collects gt semantic seg
cfg.data.val['pipeline'][2]['transforms'][3]['keys'].append('gt_semantic_seg')
cfg.data.val['pipeline'][2]['transforms'][4]['keys'].append('gt_semantic_seg')

# Remove some augmentationsfrom val pipeline
#cfg.data.val['pipeline'][2]['transforms'][1] = {'type': 'RandomFlip', 'prob': 0.0}
#cfg.data.val['pipeline'][2]['transforms'][2] = {'type': 'Resize', 'img_scale': (512, 512), 'ratio_range': (1.0, 1.0)}


datasets = [build_dataset(cfg.data.train)]
# More changes to switch to RGB-S data
datasets[0].img_suffix = '.npy'
datasets[0].seg_map_suffix = '.npy'

# Set the number of training steps (iterations)
cfg.runner['max_iters'] = 3e5


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES


# SyncBN is not support for DP
model = revert_sync_batchnorm(model)

# Create work_dir
# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'
cfg.gpu_ids = range(1)
cfg.seed = 5050
#cfg.device = get_device()

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())



usage: ipykernel_launcher.py [-h] [--config CONFIG] [--load-from LOAD_FROM]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"1060e9a4-537a-4d54-b6bf-864f51825a5b" --shell=9002 --transport="tcp" --iopub=9004 --f=/root/.local/share/jupyter/runtime/kernel-v2-411559qpytvDZCR9pY.json


SystemExit: 2